Installing dependencies

In [13]:
#%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Trying out TradingView API

In [14]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [15]:
import os
from binance.client import Client
import pandas as pd

In [20]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']

# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
client.get_symbol_info('BNBBTC')

{'symbol': 'BNBBTC',
 'status': 'TRADING',
 'baseAsset': 'BNB',
 'baseAssetPrecision': 8,
 'quoteAsset': 'BTC',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'cancelReplaceAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': False,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.00000100',
   'maxPrice': '10.00000000',
   'tickSize': '0.00000100'},
  {'filterType': 'PERCENT_PRICE',
   'multiplierUp': '5',
   'multiplierDown': '0.2',
   'avgPriceMins': 1},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.01000000',
   'maxQty': '9000.00000000',
   'stepSize': '0.01000000'},
  {'filterType': 'MIN_NOTIONAL',
   'minNotional': '0.00010000',
   'applyToMarket': True,
   'avgPriceMins': 1}